In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
import pickle
import re

In [ ]:
def dfWindowECG(df, dfList, labelList):
  index = 0
  while(index+int(60*700)<len(df)):
    dfList.append(np.array(df['c_ecg'][index:index+int(60*700)]))
    if int(df['w_label'][index])==3:
      labelList.append(2)
    else:
      labelList.append(int(df['w_label'][index]))
    index = index + int(0.25*700)
  return dfList, labelList

In [ ]:
CSV_FILE_PATH = '/content/drive/MyDrive/PhD research/Stress Detection/9/9_Class_1.csv'
dfMain = pd.read_csv(CSV_FILE_PATH)
dfMain.head()

,c_ax,c_ay,c_az,c_ecg,c_emg,c_eda,c_temp,c_resp,w_label
0,0.9128,-0.0858,-0.0858,0.172073,-0.013321,2.141571,33.780762,6.053162,1.0
1,0.9122,-0.0866,-0.0862,0.157242,0.006180,2.154922,33.774689,6.065369,1.0
2,0.9140,-0.0890,-0.0838,0.143967,0.004440,2.141571,33.723114,6.071472,1.0
3,0.9146,-0.0890,-0.0822,0.144882,-0.019180,2.159119,33.789917,6.065369,1.0
4,0.9150,-0.0878,-0.0798,0.149963,-0.027008,2.142334,33.782318,6.074524,1.0


In [ ]:
dfList = []
labelList = []

In [ ]:
filesList = glob('/content/drive/MyDrive/PhD research/Stress Detection/[5, 6, 7]/*')

for fCsv in tqdm(filesList):
  dfMain = pd.read_csv(fCsv)
  dfList, labelList = dfWindowECG(dfMain, dfList, labelList)

100%|██████████| 9/9 [00:22<00:00,  2.45s/it]


In [ ]:
len(dfList), len(labelList)

(24309, 24309)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, random_split
import torch.nn as nn

In [ ]:
class StressWESAD(Dataset):
    def __init__(self, dataList, labelList):
        self.data = dataList
        self.labelList = labelList
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        data = torch.tensor([self.data[idx]])
        data = torch.permute(data, (1, 0)).type(torch.FloatTensor)
        label = torch.tensor(self.labelList[idx] - 1) # map labels to 0,1

        return data, label

In [ ]:
DATA_STORE_PATH = '/content/drive/MyDrive/PhD research/Stress Detection'
SAVE_WEIGHT_PATH = '/content/drive/MyDrive/PhD research/Stress Detection/WESAD training/5_6_7 on top of 2_3_4 conv1d'
torch.manual_seed(0)

In [ ]:
trainDataset = StressWESAD(dfList, labelList)
lengths = [int(len(trainDataset)*0.8), len(trainDataset) - int(len(trainDataset)*0.8)]
train, valid = random_split(trainDataset, lengths)

# CNN1D model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_features = 700*60
n_classes = 2

class ConvNet1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(n_features, 64, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.layer2 = nn.Flatten()
        self.layer3 = nn.Sequential(
            nn.Linear(64,100),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Linear(100,n_classes),
            nn.Softmax())

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return out

model = ConvNet1D()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def getLastEpoch(folderName):
  '''
  Make sure last number is the epoch number
  gets input the weight path folder name
  '''
  files = glob(SAVE_WEIGHT_PATH + '/*')

  epochNumbers = []
  for fileName in files:
    numbers = re.findall(r'\d+', fileName)
    epochNumbers.append(int(numbers[-1]))

  lastUpdated = max(epochNumbers)

  for filename in reversed(files):
    if str(lastUpdated) in filename:
      return lastUpdated, fileName

In [ ]:
class ConvNet1D_training():
  def __init__(self, cnn_network, optimizer):
    self.network = cnn_network
    self.optimizer = torch.optim.Adam(self.network.parameters(), lr=1e-3)

  def train(self, loss_function, epochs, batch_size, training_set, resume):

      #  creating log
      log_dict = {
          'training_loss_per_batch': [],
          'validation_loss_per_batch': [],
          'visualizations': []
      }

      #  defining weight initialization function
      def init_weights(module):
        if isinstance(module, nn.Conv2d):
          torch.nn.init.xavier_uniform_(module.weight)
          module.bias.data.fill_(0.01)
        elif isinstance(module, nn.Linear):
          torch.nn.init.xavier_uniform_(module.weight)
          module.bias.data.fill_(0.01)

      #  initializing network weights
      self.network.apply(init_weights)

      #  creating dataloaders
      train_loader = DataLoader(training_set, batch_size=batch_size, num_workers=1, shuffle=True)
      # val_loader = DataLoader(validation_set, batch_size=10, num_workers=1, shuffle=True)
      # test_loader = DataLoader(test_set, batch_size=10, num_workers=1, shuffle=True)

      #  setting convnet to training mode
      self.network.train()
      self.network.to(device)

      if resume==True:
        lastEpoch, loadFileName = getLastEpoch(SAVE_WEIGHT_PATH)
        print(loadFileName)

        self.network.load_state_dict(torch.load(loadFileName))

      else:
        lastEpoch = 0

      for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        train_losses = []

        #------------
        #  TRAINING
        #------------
        print('training...')
        for images, labels in tqdm(train_loader):
          #  zeroing gradients
          self.optimizer.zero_grad()
          #  sending images to device
          images = images.to(device)
          #  reconstructing images
          output = self.network(images)
          #  computing loss

          loss = loss_function(output, labels)
          #  calculating gradients
          loss.backward()
          #  optimizing weights
          self.optimizer.step()

          #--------------
          # LOGGING
          #--------------
          log_dict['training_loss_per_batch'].append(loss.item())

        epochUpdated = lastEpoch + epoch

        savePath = SAVE_WEIGHT_PATH + f'/5_6_7_2_3_4_139_feb_37_{epochUpdated}.pth'
        torch.save(self.network.state_dict(), savePath)

      return log_dict

In [ ]:
trainerSetup = ConvNet1D_training(model, optimizer)
trainerSetup.train(criterion, 100, 10, train, True)

/content/drive/MyDrive/PhD research/Stress Detection/WESAD training/5_6_7 on top of 2_3_4 conv1d/5_6_7_2_3_4_139_feb_37_88.pth
Epoch 1/100
training...


  0%|          | 0/1945 [00:00<?, ?it/s]<ipython-input-9-2d309f48c7e0>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  data = torch.tensor([self.data[idx]])
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|██████████| 1945/1945 [04:02<00:00,  8.04it/s]


Epoch 2/100
training...


  0%|          | 0/1945 [00:00<?, ?it/s]<ipython-input-9-2d309f48c7e0>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  data = torch.tensor([self.data[idx]])
100%|██████████| 1945/1945 [03:32<00:00,  9.17it/s]


Epoch 3/100
training...


...

  0%|          | 0/1945 [00:00<?, ?it/s]<ipython-input-9-2d309f48c7e0>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  data = torch.tensor([self.data[idx]])
100%|██████████| 1945/1945 [04:13<00:00,  7.67it/s]


Epoch 39/100
training...


  0%|          | 0/1945 [00:00<?, ?it/s]<ipython-input-9-2d309f48c7e0>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  data = torch.tensor([self.data[idx]])
100%|██████████| 1945/1945 [04:12<00:00,  7.71it/s]


Epoch 40/100
training...


  0%|          | 0/1945 [00:00<?, ?it/s]<ipython-input-9-2d309f48c7e0>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  data = torch.tensor([self.data[idx]])
 37%|███▋      | 728/1945 [01:35<02:10,  9.33it/s]